# PROCESSING

In [1]:
import pandas as pd
import numpy as np

In [2]:
gene_df = pd.read_csv('data/gene_list.txt',header=None)
gene_list = gene_df[0].tolist()

In [3]:
#clin_patient = pd.read_csv('data/data_clinical_patient.txt', sep='\t', comment='#')
clin_sample = pd.read_csv('data/data_clinical_sample.txt', sep='\t', comment='#')
mut = pd.read_csv('data/data_mutations.txt', sep='\t', comment='#')
cna = pd.read_csv('data/data_cna.txt', sep='\t')
rna = pd.read_csv('data/data_mrna_illumina_microarray_zscores_ref_diploid_samples.txt', sep='\t', comment='#')
meth = pd.read_csv('data/data_methylation_promoters_rrbs.txt', sep='\t')

## mut 

In [4]:
print('number of genes: ',len(mut.Hugo_Symbol.unique()))
print('number of patients: ',len(mut.Tumor_Sample_Barcode.unique()))

number of genes:  173
number of patients:  2369


Select mutations involving genes that are present in the breast cancer gene list

In [5]:
mut_filter = mut[['Hugo_Symbol','Tumor_Sample_Barcode']].copy()
mut_filter = mut_filter[mut_filter['Hugo_Symbol'].isin(gene_list)]
mut_filter = mut_filter.drop_duplicates()
print('Number of mutated genes that are present in breast cancer gene list: ',len(mut_filter.Hugo_Symbol.unique()))
print('Number of patients with small mutations in the selected genes: ',len(mut_filter.Tumor_Sample_Barcode.unique()))
mut_filter

Number of mutated genes that are present in breast cancer gene list:  28
Number of patients with small mutations in the selected genes:  2135


,Hugo_Symbol,Tumor_Sample_Barcode
0,TP53,MTS-T0058
10,TP53,MTS-T0059
11,ERBB2,MTS-T0059
14,EP300,MTS-T0059
23,PIK3CA,MTS-T0061
...,...,...
17257,PIK3CA,MB-0899
17261,TP53,MB-0901
17265,PIK3CA,MB-0904
17267,PIK3CA,MB-0906


From the filtered table, create a new binary-valued matrix with patients as rows and genes as columns. Entries are 0 if no mutation is present in the selected gene for the corresponding patient, and 1 if mutations are present. No difference is reported if more than one mutation is present in the same gene.

In [6]:
mut_matrix = pd.crosstab(mut_filter['Tumor_Sample_Barcode'], mut_filter['Hugo_Symbol'])
mut_matrix.index.name = None
mut_matrix.columns.name = None
mut_matrix

,AKT1,ARID1A,ARID1B,BAP1,BRCA1,BRCA2,BRIP1,CASP8,CDH1,CDKN1B,...,NCOR1,NOTCH1,PBRM1,PIK3CA,PTEN,RB1,SMARCD1,STK11,TBX3,TP53
MB-0002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
MB-0005,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
MB-0006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
MB-0008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
MB-0010,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MTS-T2428,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
MTS-T2429,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
MTS-T2430,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
MTS-T2431,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


Check presence of missing data, change name to columns and add 'mut' column

In [7]:
# check presence of missing data
nan = mut_matrix.isnull().values.any()
print('Presence of missing data: ',nan)

# change column names
mut_matrix.columns = 'mut_' + mut_matrix.columns
# add 'mut' column indicating which patients have small mutations in the selected genes
mut_matrix['mut'] = True
mut_matrix

Presence of missing data:  False


,mut_AKT1,mut_ARID1A,mut_ARID1B,mut_BAP1,mut_BRCA1,mut_BRCA2,mut_BRIP1,mut_CASP8,mut_CDH1,mut_CDKN1B,...,mut_NOTCH1,mut_PBRM1,mut_PIK3CA,mut_PTEN,mut_RB1,mut_SMARCD1,mut_STK11,mut_TBX3,mut_TP53,mut
MB-0002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,True
MB-0005,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,True
MB-0006,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,True
MB-0008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,True
MB-0010,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MTS-T2428,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,True
MTS-T2429,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,True
MTS-T2430,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,True
MTS-T2431,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,True


## cna

In [8]:
print('number of genes: ',len(cna.Hugo_Symbol.unique()))
print('number of patients: ',len(cna.columns)-2)

number of genes:  22542
number of patients:  2173


Filtering cna table according to genes present in breast cancer genes

In [12]:
# delete 'Entrez_Gene_Id' column
cna_filter = cna.drop('Entrez_Gene_Id', axis=1).copy()
# filter rows corresponding to selected genes 
cna_filter = cna_filter[cna_filter['Hugo_Symbol'].isin(gene_list)]
print('Number of genes with copy numebr aberrations that are present in breast cancer gene list: ',len(cna_filter.Hugo_Symbol.unique()))
print('Number of patients with copy number aberrations in the selected genes: ',len(cna_filter.columns.unique())-1)

Number of genes with copy numebr aberrations that are present in breast cancer gene list:  63
Number of patients with copy number aberrations in the selected genes:  2173


Create matrix with patients as rows and genes as columns and check presence of missing data

In [15]:
# transpose index and columns
cna_filter = cna_filter.T
cna_filter.columns = cna_filter.iloc[0]
cna_filter = cna_filter.drop('Hugo_Symbol',axis=0)
cna_filter.columns.name = None
cna_filter = cna_filter.astype(int)
# check presence of missing data
nan = cna_filter.isnull().values.any()
print('Presence of missing data: ',nan)

cna_filter

Presence of missing data:  False


,AKT1,AR,ARID1A,ARID1B,ASPM,ATM,ATRIP,BAP1,BARD1,BRCA1,...,RAD54L,RB1,RRAS2,SALL4,SMARCD1,STK11,TBX3,TP53,XRCC2,ZMYM3
MB-0000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MB-0039,0,0,0,-1,0,-1,0,0,0,0,...,0,-1,0,0,0,0,0,0,-1,0
MB-0045,1,0,-1,-1,0,-1,-1,-1,0,0,...,-1,-1,0,0,-1,0,0,-1,0,0
MB-0046,0,-1,0,0,2,0,0,0,0,-1,...,0,-1,0,1,0,-1,1,-1,0,-1
MB-0048,0,1,0,0,1,-1,0,0,0,-1,...,0,0,0,0,0,-1,0,-1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-6020,0,0,0,0,1,-1,1,1,0,-1,...,0,0,0,1,0,0,0,-1,0,0
MB-6213,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MB-6230,-1,0,0,-1,1,0,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
MB-7148,0,0,0,0,1,0,-1,-1,0,0,...,0,0,0,0,0,0,0,0,0,0


Create new matrix with patients as rows and information about deletions and duplications stored in two different columns for each gene. Add 'cna' column.

In [16]:
cna_matrix = pd.DataFrame()
for i in cna_filter.columns:
    
    label_dup = 'cna_dup_'+i
    cna_dup = pd.DataFrame(np.where(cna_filter[i]>0,1,0))
    cna_dup.columns = [label_dup]

    label_del = 'cna_del_'+i
    cna_del = pd.DataFrame(np.where(cna_filter[i]<0,1,0))
    cna_del.columns = [label_del]
    
    cna_matrix = pd.concat([cna_matrix,cna_dup,cna_del],axis=1)
cna_matrix.index = cna_filter.index

# add 'cna' column indicating which patients have copy number variations in the selected genes
cna_matrix['cna'] = True
cna_matrix

C:\Users\simacri\AppData\Local\Temp\ipykernel_12868\2234565499.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cna_matrix['cna'] = True


,cna_dup_AKT1,cna_del_AKT1,cna_dup_AR,cna_del_AR,cna_dup_ARID1A,cna_del_ARID1A,cna_dup_ARID1B,cna_del_ARID1B,cna_dup_ASPM,cna_del_ASPM,...,cna_del_STK11,cna_dup_TBX3,cna_del_TBX3,cna_dup_TP53,cna_del_TP53,cna_dup_XRCC2,cna_del_XRCC2,cna_dup_ZMYM3,cna_del_ZMYM3,cna
MB-0000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
MB-0039,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,True
MB-0045,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,True
MB-0046,0,0,0,1,0,0,0,0,1,0,...,1,1,0,0,1,0,0,0,1,True
MB-0048,0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,1,1,0,1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-6020,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,True
MB-6213,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,True
MB-6230,0,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,True
MB-7148,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,True


## rna

In [17]:
print('number of genes: ',len(rna.Hugo_Symbol.unique()))
print('number of patients: ',len(rna.columns.unique())-2)

number of genes:  20387
number of patients:  1980


Filter rna table according to brast cancer gene list

In [18]:
# delete 'Entrez_Gene_Id' column
rna_filter = rna.drop('Entrez_Gene_Id', axis=1).copy()
# filter rows corresponding to selected genes 
rna_filter = rna_filter[rna_filter['Hugo_Symbol'].isin(gene_list)]
print('Number of breast cancer genes with rna expression data: ',len(rna_filter.Hugo_Symbol.unique()))
print('Number of patients with rna expression data: ',len(rna_filter.columns.unique())-1)

Number of breast cancer genes with rna expression data:  63
Number of patients with rna expression data:  1980


Verify presence of dupicated genes with different espression values and remove them. Check presence of missing data.

In [21]:
# verify presence of duplicated genes
rna_filter = rna_filter.drop_duplicates()
dupl_gene_list = []
for i in rna_filter['Hugo_Symbol'].unique():
    rna_single_gene = rna_filter.query("Hugo_Symbol=='"+i+"'")
    if (len(rna_single_gene)>1) and (len(rna_single_gene.drop_duplicates())>1):
        dupl_gene_list.append(i)
print('Genes with different expression values for each patients: ',dupl_gene_list)
# delete genes 
rna_filter.index = rna_filter.Hugo_Symbol
if len(dupl_gene_list) > 0:
    for i in dupl_gene_list:
        rna_filter = rna_filter.drop(i,axis=0)
        
# check presence of missing data
nan = rna_filter.isnull().values.any()
print('Presence of missing data: ',nan)

Genes with different expression values for each patients:  ['ARID1B']
Presence of missing data:  False


Create new matrix with patients as rows and genes as columns. Add 'rna' column.

In [22]:
rna_matrix = rna_filter.T.copy()
rna_matrix.columns = rna_matrix.iloc[0]
rna_matrix.columns.name = None
rna_matrix = rna_matrix.drop('Hugo_Symbol')
rna_matrix.columns = 'rna_' + rna_matrix.columns
# add 'rna' column indicating which patients have rna expression data
rna_matrix['rna'] = True
rna_matrix

,rna_NCOR1,rna_HGF,rna_FOXA1,rna_MSH6,rna_CDKN1B,rna_CDH1,rna_FADD,rna_RAD50,rna_BRCA2,rna_IRS4,...,rna_RB1,rna_MSH2,rna_RRAS2,rna_FBLN2,rna_ATM,rna_IKZF3,rna_TBX3,rna_CHEK2,rna_EP300,rna
MB-0362,1.0688,-0.615,0.6157,-1.1732,-0.9085,0.2545,2.0512,0.8194,0.656,-0.6345,...,0.0136,-0.7184,-0.8378,0.2111,-1.8378,-0.6141,-0.6923,-0.4167,0.6118,True
MB-0346,0.7793,0.0869,0.0284,-0.8897,-1.6337,1.797,0.3542,0.4059,1.234,-0.5464,...,-0.5372,-1.1661,0.0306,-1.3844,-2.8885,1.2891,-0.1955,1.8244,0.7471,True
MB-0386,0.1734,2.4248,0.3589,-2.4632,-1.063,-0.3024,-0.4019,0.2079,-0.6626,1.0924,...,0.6206,-1.3387,-0.1813,1.1368,-1.2912,0.1893,0.2539,-0.7337,0.5146,True
MB-0574,0.7201,-0.6548,0.2738,-2.1186,-1.4174,1.513,1.7116,1.3754,-1.0709,-1.9681,...,1.8941,-0.7967,-0.6906,-0.2951,-1.2406,-0.7358,0.7687,0.7877,0.5379,True
MB-0185,-0.2742,1.684,0.4365,0.1175,-0.7677,0.4657,0.4333,-1.1464,-0.0708,-0.218,...,0.3192,-0.546,1.3484,-1.4457,-0.7969,1.1604,0.6501,4.2078,0.7796,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5453,0.8373,-2.453,-0.3096,0.5525,1.4565,-0.1619,0.2785,0.8496,-0.5676,-0.34,...,-0.0685,0.5833,0.3933,-1.3792,1.6544,-1.188,0.6562,-0.5058,-0.5387,True
MB-5471,-0.406,-1.2909,0.3128,0.5762,0.6839,0.5722,-0.7621,-0.4317,0.7238,0.44,...,0.1549,-0.3421,-0.544,0.5705,0.7298,-0.5398,-0.0435,2.5491,1.2823,True
MB-5127,1.7295,0.9662,0.5625,0.8419,2.1601,0.7309,1.6882,-0.0815,0.3955,-0.5292,...,0.734,0.9264,-1.9107,-1.6665,0.4067,-0.1244,-0.8301,0.2351,-0.9663,True
MB-4313,-2.9275,-0.5509,-0.7498,-0.6392,-0.8716,-0.8985,-1.9611,-3.0446,0.4947,3.9558,...,-1.0741,-1.3687,-1.3555,-0.2023,-1.1483,0.3596,0.0385,-2.3226,-4.377,True


## meth 

In [23]:
print('number of genes: ',len(meth.Hugo_Symbol.unique()))
print('number of patients: ',len(meth.columns.unique())-2)

number of genes:  13184
number of patients:  1417


Filter methylation data using the breast cancer gene list.

In [33]:
meth_filter = meth[meth['Hugo_Symbol'].isin(gene_list)].copy()
meth_filter = meth_filter.T.copy()
meth_filter.columns = meth_filter.iloc[0]
meth_filter.columns.name = None
meth_filter = meth_filter.drop('Hugo_Symbol')
meth_filter.columns = 'meth_' + meth_filter.columns
print('Number of breast cancer genes with methylation data: ',len(meth_filter.columns.unique()))
print('Number of patients with methylation data: ',len(meth_filter.index.unique()))

Number of breast cancer genes with methylation data:  54
Number of patients with methylation data:  1418


Check the presence of missing data and perform imputation by substituting missing data of a selected gene with the mean value over patients of the other data corresponding to the same gene. 

In [35]:
# check presence of missing data
nan = meth_filter.isnull().values.any()
print('Presence of missing data: ',nan)
nan_tot = meth_filter.isnull().sum().sum()
perc = nan_tot*100/(len(meth_filter.columns)*len(meth_filter))
print('Percentage of nan over the total amount of data: ',perc)

Presence of missing data:  True
Percentage of nan over the total amount of data:  4.347542182521026


In [37]:
# imputation with mean value over patients
meth_matrix = meth_filter.copy()
for i in meth_matrix.columns:
    if meth_matrix[i].isnull().values.any() == False:
        continue
    mean = meth_matrix[i][meth_matrix[i].notna()].mean()
    meth_matrix[i][meth_matrix[i].isna()] = mean
nan = meth_matrix.isnull().values.any()
print('Presence of missing data after imputation: ',nan)

Presence of missing data after imputation:  False


In [38]:
# add 'meth' column indicating which patients have methylation data
meth_matrix['meth'] = True
meth_matrix

,meth_AKT1,meth_AR,meth_ARID1A,meth_ARID1B,meth_ASPM,meth_ATM,meth_ATRIP,meth_BAP1,meth_BARD1,meth_BRCA1,...,meth_RB1,meth_RRAS2,meth_SALL4,meth_SMARCD1,meth_STK11,meth_TBX3,meth_TP53,meth_XRCC2,meth_ZMYM3,meth
MB-0006,0.012346,0.345133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014706,...,0.002094,0.007246,0.374549,0.0,0.0,0.079208,0.0,0.00365,0.348101,True
MB-0028,0.042254,0.095238,0.008048,0.002353,0.011111,0.0,0.005814,0.018692,0.002066,0.0,...,0.005792,0.0,0.737374,0.0,0.0,0.058824,0.0,0.007143,0.145455,True
MB-0035,0.005587,0.15,0.0,0.001468,0.0,0.0,0.003279,0.015933,0.0,0.047583,...,0.0,0.0,0.875949,0.016287,0.0,0.0,0.0,0.0,0.507246,True
MB-0046,0.0,0.214286,0.0,0.00108,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.379421,0.0,0.0,0.0,0.0,0.004902,0.239583,True
MB-0050,0.032615,0.263158,0.00141,0.0,0.004098,0.0,0.002632,0.0,0.0,0.047583,...,0.001374,0.005063,0.572327,0.0,0.0,0.0,0.0,0.0,0.098446,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-7289,0.025,0.314815,0.001196,0.003783,0.007067,0.0,0.001653,0.0,0.002601,0.063291,...,0.0,0.0,0.695364,0.005848,0.0,0.0,0.0,0.00607,0.242915,True
MB-7291,0.0,0.30303,0.001185,0.0,0.0,0.0,0.001435,0.015933,0.002861,0.0,...,0.001116,0.0,0.579268,0.002268,0.0,0.028571,0.0,0.001408,0.201754,True
MB-7292,0.032615,0.291393,0.006309,0.007782,0.0,0.0,0.005076,0.015933,0.003623,0.02439,...,0.0,0.02509,0.568627,0.0,0.00641,0.0,0.004141,0.00995,0.21374,True
MB-7293,0.017857,0.291393,0.001594,0.0,0.0,0.009901,0.0,0.0,0.001592,0.047583,...,0.0,0.0,0.468599,0.0,0.0,0.0,0.0,0.002878,0.39196,True


## Merge all genomic data

In [122]:
matrix = mut_matrix.merge(cna_matrix, left_index=True, right_index=True, how='outer')
matrix = matrix.merge(rna_matrix, left_index=True, right_index=True, how='outer')
matrix = matrix.merge(meth_matrix, left_index=True, right_index=True, how='outer')

matrix.loc[matrix['mut'] != True, 'mut'] = False
matrix.loc[matrix['cna'] != True, 'cna'] = False
matrix.loc[matrix['rna'] != True, 'rna'] = False
matrix.loc[matrix['meth'] != True, 'meth'] = False

matrix

,mut_AKT1,mut_ARID1A,mut_ARID1B,mut_BAP1,mut_BRCA1,mut_BRCA2,mut_BRIP1,mut_CASP8,mut_CDH1,mut_CDKN1B,...,meth_RB1,meth_RRAS2,meth_SALL4,meth_SMARCD1,meth_STK11,meth_TBX3,meth_TP53,meth_XRCC2,meth_ZMYM3,meth
MB-0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
MB-0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
MB-0005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
MB-0006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002094,0.007246,0.374549,0.0,0.0,0.079208,0.0,0.00365,0.348101,True
MB-0008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MTS-T2428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
MTS-T2429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
MTS-T2430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
MTS-T2431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [83]:
print('Number of patients in:')
print('\nmut - ',len(matrix.query("mut==True"))) 
print('cna - ',len(matrix.query("cna==True"))) 
print('rna - ',len(matrix.query("rna==True"))) 
print('meth - ',len(matrix.query("meth==True"))) 

print('\nmut, cna, rna, meth - ',len(matrix.query("mut==True and cna==True and rna==True and meth==True")))

print('\nmut, cna, rna - ',len(matrix.query("mut==True and cna==True and rna==True"))) 
print('mut, cna, meth - ',len(matrix.query("mut==True and cna==True and meth==True")))
print('mut, rna, meth - ',len(matrix.query("mut==True and rna==True and meth==True")))
print('cna, rna, meth - ',len(matrix.query("cna==True and rna==True and meth==True")))

print('\nmut, cna - ',len(matrix.query("mut==True and cna==True"))) 
print('mut, rna - ',len(matrix.query("mut==True and rna==True"))) 
print('cna, rna - ',len(matrix.query("cna==True and rna==True"))) 

Number of patients in:

mut -  2135
cna -  2173
rna -  1980
meth -  1418

mut, cna, rna, meth -  1253

mut, cna, rna -  1691
mut, cna, meth -  1254
mut, rna, meth -  1253
cna, rna, meth -  1417

mut, cna -  1854
mut, rna -  1691
cna, rna -  1980


Add label indicating the tumor classification

In [123]:
matrix_clin = clin_sample.CANCER_TYPE_DETAILED.copy()
matrix_clin.index = clin_sample.PATIENT_ID
matrix = matrix.merge(matrix_clin, left_index=True, right_index=True, how='inner')
matrix

,mut_AKT1,mut_ARID1A,mut_ARID1B,mut_BAP1,mut_BRCA1,mut_BRCA2,mut_BRIP1,mut_CASP8,mut_CDH1,mut_CDKN1B,...,meth_RRAS2,meth_SALL4,meth_SMARCD1,meth_STK11,meth_TBX3,meth_TP53,meth_XRCC2,meth_ZMYM3,meth,CANCER_TYPE_DETAILED
MB-0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Breast Invasive Ductal Carcinoma
MB-0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Breast Invasive Ductal Carcinoma
MB-0005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Breast Invasive Ductal Carcinoma
MB-0006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.007246,0.374549,0.0,0.0,0.079208,0.0,0.00365,0.348101,True,Breast Mixed Ductal and Lobular Carcinoma
MB-0008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Breast Mixed Ductal and Lobular Carcinoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MTS-T2428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Invasive Breast Carcinoma
MTS-T2429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Invasive Breast Carcinoma
MTS-T2430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Invasive Breast Carcinoma
MTS-T2431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Invasive Breast Carcinoma


In [124]:
# save in file .txt
matrix['PATIENT_ID'] = matrix.index
matrix.index = np.arange(len(matrix))
matrix.to_csv('data/data_final.txt', sep='\t', index=False)
#pd.read_csv('data/data_final.txt', sep='\t')

C:\Users\simacri\AppData\Local\Temp\ipykernel_12868\1333220468.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  matrix['PATIENT_ID'] = matrix.index
